## FAKE NEWS Classification
* In this project we are classifying the News is fake or not 

###  Importing the libraries 

In [16]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

### Importing the DataSet

In [2]:
train_data = pd.read_csv('fake_train.csv')
test_data = pd.read_csv('fake_test.csv')

In [3]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


* The have the text data and we should pre-process the data before the fiting to the model
* This is a binary classification problem so we are using logistic regression for this data to classify the news is fack or not 


In [4]:
train_data.shape

(20800, 5)

In [5]:
test_data.shape

(5200, 4)

### Data Pre-processing 

In [6]:
train_data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

* There are some missing values in our data and we have to process to fil the missing value
* As we have more than 20000 data points, we can neglect these missing values 
* So we are replacing these missing values by null string 

In [7]:
# Replacing the null values to null string 

train_data = train_data.fillna('')


* We are not using the text feature for our prediction, because it has large paragraph of the news 
* It will take time to process the large data 

* We are merging the title and author features as "contents"

In [8]:
train_data['content'] = train_data['author']+' '+train_data['title']

In [9]:
train_data.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [10]:
train_data['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

### Processing the text data

#### Stemming and removing the stopwords

In [15]:
port = PorterStemmer()

In [19]:
def stem(content):
    stemmer = re.sub('[^a-zA-Z]',' ',content)
    sremmer = stemmer.lower()
    stemmer = stemmer.split()
    stemmer = [port.stem(word) for word in stemmer if not word in stopwords.words('english')]
    stemmer = ' '.join(stemmer)
    return stemmer

In [20]:
train_data['content'] = train_data['content'].apply(stem)
#train_data['text'] = train_data['text'].apply(stem)

* We are applying stem function to our data set
* Here the text feature contains large paragraphs it will take much time to convert 
* So We are using only content feature which is concatination of title and author feature to build our model

#### Seperating the Dependent and independent Feature as x and y

In [29]:
x = train_data['content'].values
y = train_data['label']

In [32]:
print (x)

['darrel lucu hou dem aid We even see comey letter jason chaffetz tweet It'
 'daniel J flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com whi truth might get fire' ...
 'michael J de la merc rachel abram maci Is said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia To hold parallel exerci In balkan'
 'david swanson keep F aliv']


In [33]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 20800, dtype: int64

* In x have a text data and we should process that text data ,because ML algorithm will not understand the text data 

### converting the text data to numerical data

In [34]:
vector = TfidfVectorizer()
vector.fit(x)

x = vector.transform(x)

In [36]:
print (x) 

  (0, 16397)	0.20950904108906998
  (0, 15581)	0.27362569473215625
  (0, 13373)	0.2464783903145084
  (0, 8836)	0.34926835263542055
  (0, 8560)	0.28061353784019905
  (0, 7626)	0.23808523001881335
  (0, 7536)	0.18266339610127144
  (0, 6942)	0.21012186616975223
  (0, 4929)	0.22412278288507764
  (0, 3768)	0.2598725040309882
  (0, 3576)	0.34571179155648435
  (0, 2942)	0.23711773710608378
  (0, 2473)	0.3531641245177442
  (0, 264)	0.2594575292258874
  (1, 16690)	0.3025156488372128
  (1, 6756)	0.19152496072048605
  (1, 5451)	0.7186013955384664
  (1, 3544)	0.2653147533915268
  (1, 2799)	0.19208753385709676
  (1, 2215)	0.36915639258038363
  (1, 1885)	0.15614790568229528
  (1, 1489)	0.2957471154505952
  (2, 16518)	0.3283886058850443
  (2, 15506)	0.39240991457480234
  (2, 9549)	0.4661459269188732
  :	:
  (20797, 10234)	0.07648433195119052
  (20797, 9517)	0.17295096754746545
  (20797, 9448)	0.2911126156876309
  (20797, 8917)	0.358288889990968
  (20797, 8296)	0.22117650239990858
  (20797, 7509)	0.140

#### Splitting the data Set to training ang testing

In [37]:
from sklearn.model_selection import train_test_split

In [39]:
x_train,x_test,y_train,y_test = train_test_split(x,y,stratify=y, test_size=0.2)

### Building a Model

In [40]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

In [41]:
model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Evaluating the model

#### Accuracy score

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
# accuracy score for training data
prediction_1 = model.predict(x_train)
accuracy_1 = accuracy_score(prediction_1,y_train)

print('Accuracy score of the training data : ',accuracy_1)

Accuracy score of the training data :  0.9878004807692308


In [44]:
# accuracy score for testing data

prediction_2 = model.predict(x_test)
accuracy_2 = accuracy_score(prediction_2,y_test)

print('Accuracy score of the training data : ',accuracy_2)

Accuracy score of the training data :  0.9762019230769231


In [61]:
X_new = x_test[3]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real
